In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Reshape, Softmax, Layer, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, ResNet50, EfficientNetB0

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory("data/train",
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode='categorical',
                                               subset='training')

val_data = val_datagen.flow_from_directory("data/train",
                                           target_size=(224, 224),
                                           batch_size=32,
                                           class_mode='categorical',
                                           subset='validation')

test_data = test_datagen.flow_from_directory("data/test",
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode='categorical',
                                             shuffle=False)

input_shape = (224, 224, 3)
num_classes = train_data.num_classes

Found 2297 images belonging to 4 classes.
Found 573 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [3]:
#CNN

model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation="softmax")
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
72/72 [==============================] - 24s 271ms/step - loss: 2.7661 - accuracy: 0.4802 - val_loss: 9.5807 - val_accuracy: 0.2880
Epoch 2/10
72/72 [==============================] - 17s 229ms/step - loss: 0.9405 - accuracy: 0.6413 - val_loss: 11.1500 - val_accuracy: 0.2880
Epoch 3/10
72/72 [==============================] - 18s 245ms/step - loss: 0.8943 - accuracy: 0.6387 - val_loss: 7.7676 - val_accuracy: 0.2914
Epoch 4/10
72/72 [==============================] - 18s 243ms/step - loss: 0.8341 - accuracy: 0.6652 - val_loss: 3.6061 - val_accuracy: 0.3508
Epoch 5/10
72/72 [==============================] - 17s 233ms/step - loss: 0.7814 - accuracy: 0.6839 - val_loss: 5.2824 - val_accuracy: 0.2949
Epoch 6/10
72/72 [==============================] - 17s 233ms/step - loss: 0.7411 - accuracy: 0.6939 - val_loss: 3.9133 - val_accuracy: 0.2042
Epoch 7/10
72/72 [==============================] - 17s 235ms/step - loss: 0.7169 - accuracy: 0.7088 - val_loss: 2.1793 - val_accuracy: 0.357

In [4]:
#Hybrid

class SelfAttention(Layer):
    def __init__(self, embed_dim):
        super(SelfAttention, self).__init__()
        self.query_dense = Dense(embed_dim)
        self.key_dense = Dense(embed_dim)
        self.value_dense = Dense(embed_dim)
        self.softmax = Softmax(axis=-1)

    def call(self, inputs):
        Q = self.query_dense(inputs)
        K = self.key_dense(inputs)
        V = self.value_dense(inputs)

        attention_scores = tf.matmul(Q, K, transpose_b=True)
        attention_weights = self.softmax(attention_scores)

        return tf.matmul(attention_weights, V)
    
def build_hybrid_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3,3), activation="relu", padding="same")(inputs)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(128, (3,3), activation="relu", padding="same")(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(128, activation="relu")(x)
    x = Reshape((1, 128))(x)

    x = SelfAttention(128)(x)
    x = Flatten()(x)

    x = Dense(64, activation="relu")(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs, outputs)
    return model

model = build_hybrid_model(input_shape, num_classes)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
72/72 [==============================] - 18s 237ms/step - loss: 1.3057 - accuracy: 0.3592 - val_loss: 1.3261 - val_accuracy: 0.3857
Epoch 2/10
72/72 [==============================] - 17s 235ms/step - loss: 1.1756 - accuracy: 0.4697 - val_loss: 1.4444 - val_accuracy: 0.2897
Epoch 3/10
72/72 [==============================] - 17s 239ms/step - loss: 1.1158 - accuracy: 0.5007 - val_loss: 1.4028 - val_accuracy: 0.3019
Epoch 4/10
72/72 [==============================] - 17s 236ms/step - loss: 1.0367 - accuracy: 0.5472 - val_loss: 1.5175 - val_accuracy: 0.3281
Epoch 5/10
72/72 [==============================] - 17s 240ms/step - loss: 0.9680 - accuracy: 0.5812 - val_loss: 1.4198 - val_accuracy: 0.3892
Epoch 6/10
72/72 [==============================] - 18s 250ms/step - loss: 0.9843 - accuracy: 0.5655 - val_loss: 1.3011 - val_accuracy: 0.3979
Epoch 7/10
72/72 [==============================] - 19s 263ms/step - loss: 0.9016 - accuracy: 0.6160 - val_loss: 1.4520 - val_accuracy: 0.4328

In [5]:
#DenseNet121

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers[:-31]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation="softmax")(x)    

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
72/72 [==============================] - 26s 290ms/step - loss: 0.9109 - accuracy: 0.6374 - val_loss: 0.7716 - val_accuracy: 0.6736
Epoch 2/10
72/72 [==============================] - 19s 259ms/step - loss: 0.4819 - accuracy: 0.8241 - val_loss: 0.5768 - val_accuracy: 0.7696
Epoch 3/10
72/72 [==============================] - 20s 270ms/step - loss: 0.3704 - accuracy: 0.8694 - val_loss: 0.4970 - val_accuracy: 0.8150
Epoch 4/10
72/72 [==============================] - 19s 265ms/step - loss: 0.2938 - accuracy: 0.8938 - val_loss: 0.3825 - val_accuracy: 0.8656
Epoch 5/10
72/72 [==============================] - 19s 257ms/step - loss: 0.2380 - accuracy: 0.9208 - val_loss: 0.3974 - val_accuracy: 0.8569
Epoch 6/10
72/72 [==============================] - 18s 253ms/step - loss: 0.2238 - accuracy: 0.9116 - val_loss: 0.4078 - val_accuracy: 0.8569
Epoch 7/10
72/72 [==============================] - 19s 258ms/step - loss: 0.1691 - accuracy: 0.9434 - val_loss: 0.4149 - val_accuracy: 0.8586

In [6]:
#ResNet50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers[:140]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
72/72 [==============================] - 24s 278ms/step - loss: 1.1344 - accuracy: 0.4980 - val_loss: 1.5019 - val_accuracy: 0.2862
Epoch 2/10
72/72 [==============================] - 18s 252ms/step - loss: 0.9914 - accuracy: 0.5629 - val_loss: 1.3836 - val_accuracy: 0.3002
Epoch 3/10
72/72 [==============================] - 18s 254ms/step - loss: 0.9202 - accuracy: 0.6173 - val_loss: 1.1741 - val_accuracy: 0.4188
Epoch 4/10
72/72 [==============================] - 19s 258ms/step - loss: 0.8912 - accuracy: 0.6243 - val_loss: 1.0933 - val_accuracy: 0.5236
Epoch 5/10
72/72 [==============================] - 19s 260ms/step - loss: 0.8193 - accuracy: 0.6622 - val_loss: 1.0952 - val_accuracy: 0.5602
Epoch 6/10
72/72 [==============================] - 19s 260ms/step - loss: 0.7941 - accuracy: 0.6626 - val_loss: 1.0679 - val_accuracy: 0.5689
Epoch 7/10
72/72 [==============================] - 18s 253ms/step - loss: 0.8037 - accuracy: 0.6565 - val_loss: 1.8302 - val_accuracy: 0.4241

In [7]:
#EfficientNetB0

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers[:-50]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
72/72 [==============================] - 23s 255ms/step - loss: 1.3929 - accuracy: 0.2860 - val_loss: 1.4047 - val_accuracy: 0.2880
Epoch 2/10
72/72 [==============================] - 17s 240ms/step - loss: 1.3622 - accuracy: 0.2978 - val_loss: 1.3980 - val_accuracy: 0.2862
Epoch 3/10
72/72 [==============================] - 18s 242ms/step - loss: 1.3546 - accuracy: 0.3047 - val_loss: 1.3613 - val_accuracy: 0.2862
Epoch 4/10
72/72 [==============================] - 18s 243ms/step - loss: 1.3484 - accuracy: 0.2912 - val_loss: 1.3621 - val_accuracy: 0.2862
Epoch 5/10
72/72 [==============================] - 17s 239ms/step - loss: 1.3330 - accuracy: 0.2995 - val_loss: 1.4096 - val_accuracy: 0.2862
Epoch 6/10
72/72 [==============================] - 17s 240ms/step - loss: 1.3209 - accuracy: 0.3239 - val_loss: 1.3492 - val_accuracy: 0.2862
Epoch 7/10
72/72 [==============================] - 17s 240ms/step - loss: 1.3066 - accuracy: 0.3287 - val_loss: 1.4567 - val_accuracy: 0.3403